# **PRAKTIKUM 6**

In [2]:
# =============================================
# 🔍 Perbandingan Exact NN, Annoy, HNSW, FAISS (Optimized)
# =============================================

import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1️⃣ Load dataset
# -------------------------------
df = pd.read_csv('dataset/songs_with_attributes_and_lyrics.csv')

features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

# Gunakan subset dulu biar cepat (misal 5000 lagu)
X = X[:5000]

# -------------------------------
# 2️⃣ Standarisasi fitur
# -------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors
sample_queries = X_scaled[:100]  # cuma ambil 100 data untuk test query

# -------------------------------
# 3️⃣ Exact Nearest Neighbor (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(sample_queries)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# 4️⃣ Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)

# build tree (10-20 cukup untuk balancing speed vs accuracy)
index_annoy.build(10)

# query cuma sebagian
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in sample_queries]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# 5️⃣ HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_scaled.shape[1])
p_hnsw.init_index(max_elements=X_scaled.shape[0], ef_construction=100, M=12)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(100)

idx_hnsw, dist_hnsw = p_hnsw.knn_query(sample_queries, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# 6️⃣ FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], 50, faiss.METRIC_L2)  # ← pakai posisi, bukan keyword

index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 5

dist_faiss, idx_faiss = index_faiss.search(sample_queries, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")


# -------------------------------
# 7️⃣ Bandingkan hasil untuk satu lagu
# -------------------------------
print("\nTop-5 neighbors for first song sample:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")


Exact NN done in 4.428 s
Annoy done in 0.059 s
HNSW done in 0.052 s
FAISS IVF done in 0.022 s

Top-5 neighbors for first song sample:
Exact NN: [   0 3836 2788 4042 2519]
Annoy:    [0, 3836, 2788, 4042, 2519]
HNSW:     [   0 3836 2788 4042 2519]
FAISS:    [   0 3836 2788 4042 2519]
